# EDA 1

## Install Libs

In [ ]:
# see the versions of your installed libs
!pip list

In [ ]:
!pip install pgeocode

In [ ]:
!pip install -U folium

In [ ]:
# !python -m pip install -U folium

In [ ]:
# upgrade your pandas
!pip install -U pandas

## Import Libs

In [ ]:
import pandas as pd

In [ ]:
# see the upgraded version
pd.__version__

## Step 1: Load Data

In [ ]:
# bolog data, cleaned
dfb = pd.read_csv('https://raw.githubusercontent.com/foxdocs/dat-bi/refs/heads/main/Data/boligclean2.csv')

In [ ]:
# check it
dfb.shape

In [ ]:
# restaurant data, uncleaned
dfr = pd.read_csv('https://raw.githubusercontent.com/foxdocs/dat-bi/refs/heads/main/Data/tripadvisor-denmark.csv')

In [ ]:
dfr.shape

### dfb

In [ ]:
dfb.columns

In [ ]:
dfb.dtypes

In [ ]:
dfb.info()

In [ ]:
dfb.sample(5)

In [ ]:
dfb.isna().sum()

In [ ]:
dfb.duplicated().sum()

Needs some extra cleaning. I will 
- remove dublicates
- remove boligid
- deal with missing values
- clean liggetid
- concat the address fields in one column and use it for geolocation
- show on the map

## Step 2: Data Preparation

### Cleaning

In [ ]:
dfb.drop_duplicates()

In [ ]:
dfb = dfb.drop('bolig_id', axis=1)
dfb.columns

In [ ]:
dfb.isna().sum()

#### Cleaning _vaercat_

In [ ]:
# fill in vaercat
dfb.plot.scatter(y="vaercat", x="kvmnum")
dfb.plot.scatter(x="vaercat", y="grundnum")

In [ ]:
dfb.vaercat.dtypes

In [ ]:
dfb["vaercat"] = dfb["vaercat"].interpolate(method='linear')

In [ ]:
dfb["vaercat"].isna().sum()

In [ ]:
dfb.sample(4)

#### Cleaning _liggetid_

In [ ]:
dfb.liggetid.dtypes

In [ ]:
# getting the digits and the dot between
dfb['liggetid'] = dfb.liggetid.str.extract('(\d+)').astype('Int64')

In [ ]:
dfb.dtypes

In [ ]:
dfb.liggetid.isna().sum()

In [ ]:
dfb['liggetid'] = dfb.liggetid.fillna(0)

In [ ]:
dfb.sample(3)

In [ ]:
dfb.liggetid.describe()

In [ ]:
dfb.columns

In [ ]:
dfb.sample()

#### Create Address

In [ ]:
dfb['address'] = dfb['vejtot'] + ' ' + dfb['zipcode'].apply(str) + ' ' + dfb['postnr'] 

In [ ]:
dfb = dfb.drop(['vejtot', 'vejname', 'streetnr'], axis=1)

In [ ]:
dfb.columns

In [ ]:
# new order the columns
order = ['address','aldernum','kvmnum','grundnum','vaercat','energicat','liggetid','zipcode','postnr','kommunenavn','prisnum'] 

In [ ]:
# reorder
dfb = dfb[order]

In [ ]:
dfb.head()

### Save in File

In [ ]:
dfb.to_csv("clean-dfb.csv", index=False)

## Visualise on Map

In [ ]:
import pgeocode  # for operations with the address
import folium    # for geovisualisation

In [ ]:
tab = pd.read_csv("clean-dfb.csv")

In [ ]:
tab.sample(2)

In [ ]:
# Load Denmark
nomi = pgeocode.Nominatim('dk')
nomi

In [ ]:
# nomi test 
nomi.query_postal_code("4736")

In [ ]:
# result = [f(x) for x in df['col']]
# result = [rd(comp) for comp in compi['comp']]

In [ ]:
# from zipcode - region
tab['region'] = [nomi.query_postal_code(x).state_name  for x in tab['zipcode']]

In [ ]:
tab['region'].unique()

In [ ]:
# from zipcode - commune
tab['county'] = [nomi.query_postal_code(x).county_name  for x in tab['zipcode']]

In [ ]:
tab['county'].unique()

In [ ]:
tab['county'] = tab.county.fillna(tab['postnr'])

In [ ]:
# from zipcode - geoloc
tab['latitude'] = [nomi.query_postal_code(x).latitude  for x in tab['zipcode']]

In [ ]:
tab['longitude'] = [nomi.query_postal_code(x).longitude  for x in tab['zipcode']]

In [ ]:
tab.isna().sum()

In [ ]:
tab = tab.dropna(subset=["region"])

In [ ]:
tab.head()

In [ ]:
tabmap = tab.groupby('zipcode').agg({'latitude':'min','longitude':'min', 'kommunenavn':'min', 'address':'count'})

In [ ]:
tabmap

### Map
Folium is a data visualisatuion library with a support for images, video, GeoJSON and TopoJSON data formats.

In [ ]:
import folium

In [ ]:
# Mark to companies'location
mapa = folium.Map(location=[55,12], tiles="OpenStreetMap", zoom_start=6)

In [ ]:
# icon_image = '/Users/tdi/Documents/Teaching/BI/2026/Images/home.png'
# icon = folium.CustomIcon(icon_image,icon_size=row['address']) 

In [ ]:
for i,row in tabmap.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['kommunenavn']).add_to(mapa)

In [ ]:
mapa